In [19]:
# Useful links: 
# https://developers.google.com/optimization/mip/mip_example

In [20]:
#!pip install ortools

In [21]:
# utility file used to generate mzn data files from the dat files
import numpy as np
# open the file in Instances folder
f = open("Instances/inst05.dat", "r")
# the first line is the number of couriers
m = int(f.readline())
# the second line is the number of items
n = int(f.readline())
# the third line is the load size of each courier
load_size = [int(x) for x in f.readline().split()]
# the fourth line is the size of each item
item_size = [int(x) for x in f.readline().split()]
# the rest is the distance matrix
distance = []
for i in range(n+1):
    distance.append([int(x) for x in f.readline().split()])
# close the file
f.close()
print("couriers:", m)
print("items:", n)
print("load_size:", load_size)
print("item_size:", item_size)
# output the distance matrix as a numpy array
distance = np.array(distance)
print("distance:\n", distance)

couriers: 2
items: 3
load_size: [18, 30]
item_size: [20, 17, 6]
distance:
 [[ 0 21 86 99]
 [21  0 71 80]
 [92 71  0 61]
 [59 80 61  0]]


In [25]:
from ortools.linear_solver import pywraplp

def solve_courier_problem(courier_capacities, item_sizes, item_distances):
    num_couriers = len(courier_capacities)
    num_items = len(item_sizes)

    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Create variables
    assignment = [[[solver.IntVar(0, 1, f'assignment_{i}_{j}_{k}') for j in range(num_items + 1)] for k in range(num_items + 1)] for i in range(num_couriers)]
    print('assignment =', assignment)
    
    # The diagonal of the matrix is 0
    solver.Add(sum(assignment[i][j][j] for i in range(num_couriers) for j in range(num_items + 1)) == 0)
    
    # Create constraints: each courier carries at least one item
    for i in range(num_couriers):
        solver.Add(sum(assignment[i][num_items][k] for k in range(num_items + 1)) >= 1)
    
    # Create constraints: items are assigned to at most one courier
    for j in range(num_items):
        solver.Add(sum(assignment[i][j][k] for i in range(num_couriers) for k in range(num_items + 1)) == 1)
    
    for j in range(num_items):
        solver.Add(sum(assignment[i][k][j] for i in range(num_couriers) for k in range(num_items + 1)) == 1)
    
    # Create constraint: n arcs in, n arcs out
    for i in range(num_couriers):
        for j in range(num_items + 1):
            solver.Add(sum(assignment[i][j][k] for k in range(num_items + 1)) == sum(assignment[i][k][j] for k in range(num_items + 1)))
    
    
    # Create constraint: courier capacities are respected
    for i in range(num_couriers):
        solver.Add(sum(assignment[i][j][k] * item_sizes[j] for j in range(num_items) for k in range(num_items + 1)) <= courier_capacities[i])
    
    # I create a variable max distance to minimize
    # the three lines below are a good beginning, but we want to also change the order of the items in the path so we need a way to express that
    max_distance = solver.NumVar(0, solver.infinity(), 'max_distance')
    for i in range(num_couriers):
        solver.Add(sum(assignment[i][j][k] * item_distances[j][k] for j in range(num_items + 1) for k in range(num_items + 1)) <= max_distance)
    
    # Create constraint: Eliminate subtours using the Miller-Tucker-Zemlin (MTZ) formulation
    for i in range(num_couriers):
        # Count the number of items assigned to this courier
        num_assigned_items = sum(assignment[i][j][k].solution_value() for j in range(num_items) for k in range(num_items + 1))
        # If the number of assigned items is greater than 1, apply subtour elimination constraint
        if num_assigned_items > 1:
            for j in range(0, num_items + 1):
                for k in range(0, num_items + 1):
                    if j != k:
                        solver.Add(assignment[i][j][k] + assignment[i][k][j] <= 1)
    
    solver.Minimize(max_distance)
    
    solver.set_time_limit(300000)
    #solver.EnableOutput()
    
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        for i in range(num_couriers):
            for j in range(num_items):
                for k in range(num_items + 1):
                    if assignment[i][j][k].solution_value() > 0:
                        print(f'Courier {i} takes item {j}')
        print('Max distance:', max_distance.solution_value())
        print('Objective value:', solver.Objective().Value())
        # I print the values of assignment
        # for each courier i create and print a item matrix
        print()
        for i in range(num_couriers):
            print(f'Courier {i} item matrix:')
            for j in range(num_items + 1):
                for k in range(num_items + 1):
                    print(int(assignment[i][j][k].solution_value()), end=' ')
                print()
            print()
        print(item_distances)
    else:
        print('The problem does not have an optimal solution.')

solve_courier_problem(load_size, item_size, distance)


assignment = [[[assignment_0_0_0, assignment_0_1_0, assignment_0_2_0, assignment_0_3_0], [assignment_0_0_1, assignment_0_1_1, assignment_0_2_1, assignment_0_3_1], [assignment_0_0_2, assignment_0_1_2, assignment_0_2_2, assignment_0_3_2], [assignment_0_0_3, assignment_0_1_3, assignment_0_2_3, assignment_0_3_3]], [[assignment_1_0_0, assignment_1_1_0, assignment_1_2_0, assignment_1_3_0], [assignment_1_0_1, assignment_1_1_1, assignment_1_2_1, assignment_1_3_1], [assignment_1_0_2, assignment_1_1_2, assignment_1_2_2, assignment_1_3_2], [assignment_1_0_3, assignment_1_1_3, assignment_1_2_3, assignment_1_3_3]]]
Solution:
Courier 0 takes item 1
Courier 1 takes item 0
Courier 1 takes item 2
Max distance: 206.0
Objective value: 206.0

Courier 0 item matrix:
0 0 0 0 
0 0 0 1 
0 0 0 0 
0 1 0 0 

Courier 1 item matrix:
0 0 1 0 
0 0 0 0 
0 0 0 1 
1 0 0 0 

[[ 0 21 86 99]
 [21  0 71 80]
 [92 71  0 61]
 [59 80 61  0]]
